In [12]:
database_path = r'E:\Research Databases\bad_actors_Coronavirus_Project_POI_Followers_30-05-20.db'
topic_map_file_path = r'C:\Users\Eli Wolfson\Documents\University\Research\FakeNews\data\twitter_over_time\twitter_healthcare_workers_11-05\tweet_topic_map.json'
country_map_file_path = r'C:\Users\Eli Wolfson\Documents\University\Research\FakeNews\data\twitter_location_analysis\POI_Followers_30-05-20\tweet_countries.json'
state_map_file_path = r'C:\Users\Eli Wolfson\Documents\University\Research\FakeNews\data\twitter_location_analysis\POI_Followers_30-05-20\tweet_states.json'
state_code_name_path = r'C:\Users\Eli Wolfson\Documents\University\Research\FakeNews\data\twitter_location_analysis\POI_Followers_30-05-20\state_code_name.json'
output_path = r'C:\Users\Eli Wolfson\Documents\University\Research\FakeNews\data\twitter_location_analysis\POI_Followers_30-05-20\topic_location_analysis'

In [13]:
import os

if not os.path.exists(output_path):
    os.mkdir(output_path)

In [14]:
import json

with open(topic_map_file_path, 'r') as file_handle:
    tweet_topic_map = json.load(file_handle)

with open(country_map_file_path, 'r') as file_handle:
    tweet_country_map = json.load(file_handle)

with open(state_map_file_path, 'r') as file_handle:
    tweet_state_map = json.load(file_handle)
    
with open(state_code_name_path, 'r') as file_handle:
    state_code_name = json.load(file_handle)

In [20]:
import pandas as pd

tweets_with_countries_topic_map = {tweet_id : tweet_topic_map[tweet_id] for tweet_id in set(tweet_country_map.keys()) & set(tweet_topic_map.keys())}
tweets_with_states_topic_map = {tweet_id : tweet_topic_map[tweet_id] for tweet_id in set(tweet_state_map.keys()) & set(tweet_topic_map.keys())}

topic_country_count_map_list = []
topic_state_count_map_list = []
for topic in range(30):
    countries = {}
    topic_tweets_with_countries = [tweet_id for (tweet_id, tweet_topic_prob) in tweets_with_countries_topic_map.items() if eval(tweet_topic_prob)[0] == topic]
    for tweet_id in topic_tweets_with_countries:
        countries[tweet_country_map[tweet_id]] = countries.get(tweet_country_map[tweet_id], 0) + 1
    topic_country_count_map_list.append(countries)
    
    states = {}
    topic_tweets_with_states = [tweet_id for (tweet_id, tweet_topic_prob) in tweets_with_states_topic_map.items() if eval(tweet_topic_prob)[0] == topic]
    for tweet_id in topic_tweets_with_states:
        states[state_code_name[tweet_state_map[tweet_id]]] = states.get(state_code_name[tweet_state_map[tweet_id]], 0) + 1
    topic_state_count_map_list.append(states)

In [21]:
countries_df = pd.DataFrame(topic_country_count_map_list, index=['t{}'.format(i) for i in range(30)])
countries_df.to_csv('{}/country_topics.csv'.format(output_path))

states_df = pd.DataFrame(topic_state_count_map_list, index=['t{}'.format(i) for i in range(30)])
states_df.to_csv('{}/state_topics.csv'.format(output_path))

In [22]:
countries_df_percentage = countries_df.divide(countries_df.sum(), axis=1)
countries_df.to_csv('{}/country_topics_percentage.csv'.format(output_path))

states_df_percentage = states_df.divide(states_df.sum(), axis=1)
states_df.to_csv('{}/state_topics_percentage.csv'.format(output_path))

In [70]:
def get_most_discussed_topics(df, country_name):
    values_list = df[country_name].sort_values(ascending=False)
    return [(values_list.index[i], values_list[i]) for i, entry in enumerate(values_list)]

In [78]:
def get_countries_topic_most_discussed(topic_id):
    return [country for country in countries_df.columns if get_most_discussed_topics(countries_df, country)[0][0] == 't{}'.format(topic_id)]

def get_states_topic_most_discussed(topic_id):
    return [state for state in states_df.columns if get_most_discussed_topics(states_df, state)[0][0] == 't{}'.format(topic_id)]